In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import cv2
from PIL import Image 
from tqdm import tqdm
from numba import jit, njit, prange, cuda
import yaml
from scipy.sparse import csr_matrix
import pickle
import os

In [34]:
@njit(nogil=True)
def norm(x_i, x_j):
    return np.sum((x_i-x_j)**2)

@njit(nogil=True)
def QUBO_matr(img):
    """
    Find QUBO matrix  for image segmentation
    img - RGB of image pixels 
    """
    h, v, c = img.shape
    N = h*v
    x = img.reshape((N, c))/255
    qubo = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
                qubo[i, i] -= norm(x[i], x[j])
                qubo[i, j] = norm(x[i], x[j])
    return qubo

@njit(nogil=True)
def QUBO_matr_sparse(img, K, decay = True):
    """
    Find QUBO matrix  for image recognition
    img - RGB of image pixels 
    K - K nearest neighbours 
    """
    h, v, c = img.shape
    N = h*v
    x = img.reshape((N, c))/255
    qubo = np.zeros((N, N))
    for i in range(h):
        for j in range(v):
            for k in range(-K, K+1):
                for l in range(-K, K+1):
                    if k != 0 or l !=0:
                        kl_square = k**2 + l**2
                        if i + k >= 0 and i + k < h and j + l >= 0 and j + l < v and kl_square <= K**2:
                            flat_ij = i*v+j
                            flat_neigh = (i+k)*v+j+l
                            if decay:
                                norm_neigh = norm(x[flat_ij], x[flat_neigh])/kl_square
                            else:
                                norm_neigh = norm(x[flat_ij], x[flat_neigh]) 
                            #print(i, j, k, l, norm_neigh)
                            qubo[flat_ij, flat_ij] -= norm_neigh
                            qubo[flat_neigh, flat_neigh] -= norm_neigh
                            qubo[flat_ij, flat_neigh] += norm_neigh
                            qubo[flat_neigh, flat_ij] += norm_neigh
    return qubo

@njit(nogil=True)
def QUBO_matr_full(img, K1, K2, B, decay = False):
    """
    Find QUBO matrix  for omage recognition
    img - RGB of image pixels 
    K1 - K1 nearest neighbours 
    """
    h, v, c = img.shape
    N = h*v
    x = img.reshape((N, c))/255
    qubo = np.zeros((N, N))
    for i in range(h):
        for j in range(v):
            for k in range(-K1, K1+1):
                for l in range(-K1, K1+1):
                    if k != 0 or l !=0:
                        kl_square = k**2 + l**2
                        if i + k >= 0 and i + k < h and j + l >= 0 and j + l < v and kl_square <= K1**2:                                
                            flat_ij = i*v+j
                            flat_neigh = (i+k)*v+j+l
                            if decay:
                                norm_neigh = norm(x[flat_ij], x[flat_neigh])/kl_square
                            else:
                                norm_neigh = norm(x[flat_ij], x[flat_neigh]) 
                            if kl_square <= K2**2:
                                qubo[flat_ij, flat_ij] += B
                                qubo[flat_neigh, flat_neigh] += B                          
                            #print(i, j, k, l, norm_neigh)
                            qubo[flat_ij, flat_ij] -= norm_neigh
                            qubo[flat_neigh, flat_neigh] -= norm_neigh
                            qubo[flat_ij, flat_neigh] += norm_neigh
                            qubo[flat_neigh, flat_ij] += norm_neigh
    return qubo

In [ ]:
K1 = n

In [4]:
def qboard_problem(problem_id, problem_type, task_number, matrix_file, where = 'MIN'):
    obj_aux = dict(to = where, filenames = matrix_file)
    return dict(id = problem_id, type = problem_type, 
                data_dir = '~/task_' + str(task_number) + '/pkl',
                #data_dir = '~/pkl',
                objective = obj_aux)
def compile_yaml(all_problems, output_file):
    output = []
    for problem in all_problems:
        output.append(qboard_problem(*problem))
    stream = open(output_file, 'w')
    yaml.dump(dict(problems = output), stream, sort_keys = False)
    stream.close()
    return

def make_pkl(matrix, filename):
    stream = open(filename, 'wb')
    pickle.dump(csr_matrix(matrix), stream)
    stream.close()
    return

In [33]:
#draw a big pictures with one tone 
folder = "imgs/big"
K1 = 8
K2 = 5
B = 0.5
decay = False
problems = []
syaml = "big.yaml"
for i, file in tqdm(enumerate(os.listdir(folder))):
    s = file[:-4]
    img = cv2.imread(folder+"/"+s+'.png')
    img1 = cv2.resize(img, (100, 100))#resize because 200*200 is too long to wait and kernel is died simultanously
    
    Q = QUBO_matr_full(img1, K1 = K1, K2 = K2, B = B, decay = decay)
    s1 = s + str(K1) +'_'+str(K2)+ '_'+str(B1) 
    spkl = s1+'.pkl' 
    problems.append([s1, 'QUBO', 3, spkl])
    make_pkl(Q, 'pkl/' + spkl)

compile_yaml(problems, syaml)

3it [00:03,  1.17s/it]


In [36]:
#draw a big pictures with negative tone and precise contour 
folder = "imgs/negative"
K1 = 10
K2 = 7
B = 1
decay = False
problems = []
syaml = "neg.yaml"
for i, file in tqdm(enumerate(os.listdir(folder))):
    s = file[:-4]
    img = cv2.imread(folder+"/"+s+'.png')
    img1 = cv2.resize(img, (120, 120))#resize because 200*200 is too long to wait and kernel is died simultanously
    
    Q = QUBO_matr_full(img1, K1 = K1, K2 = K2, B = B, decay = decay)
    s1 = s + str(K1) +'_'+str(K2)+ '_'+str(B1) 
    spkl = s1+'.pkl' 
    problems.append([s1, 'QUBO', 3, spkl])
    make_pkl(Q, 'pkl/' + spkl)

compile_yaml(problems, syaml)

3it [00:10,  3.47s/it]


In [40]:
problems = []
folder = "imgs"
decay = False
s = '2_small_c'
img = cv2.imread(folder+"/"+s+'.png')
K1 = 8
K2 = 7
syaml = 'yaml/'+s+'.yaml'
for B in range(1, 2):
    B1 = 0.8*B
    Q = QUBO_matr_full(img, K1 = K1, K2 = K2, B = B1, decay = decay)
    s1 = s + str(K1) +'_'+str(K2)+ '_'+str(B1) 
    spkl = s1+'.pkl' 
    problems.append([s1, 'QUBO', 3, spkl])
    make_pkl(Q, 'pkl/' + spkl)

compile_yaml(problems, syaml)